In [142]:
import pandas as pd
import numpy as np

In [143]:
Parkhäuser = pd.read_csv("../Final_Data/Cleaned/1000014cleaned.csv")
Flugbewegungen = pd.read_csv("../Final_Data/Cleaned/Airport_traffic_hourly_cleaned.csv")
Gasverbrauch = pd.read_csv("../Final_Data/Cleaned/Gasverbrauch_cleaned.csv")
#Schallmessungen = pd.read_csv("../Final_Data/Cleaned/Schallmessungen_cleaned.csv", sep=";") -> zu viele Datenpunkte fehlen
Stromverbrauch = pd.read_csv("../Final_Data/Cleaned/Stromverbrauch_Basel_hourly_cleaned.csv")
Wetterdaten = pd.read_csv("../Final_Data/Cleaned/Wetterdaten_Basel_cleaned22_24.csv")
Verkehrsdaten = pd.read_csv("../Final_Data/Cleaned/1000006cleaned.csv")

In [144]:

Parkhäuser.rename(columns={"RoundedTime": "Datum"}, inplace=True)
Parkhäuser.head()
#Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.head()
#Gasverbrauch.head()
#Schallmessungen.drop(columns=["Unnamed: 0"], inplace=True)
#Schallmessungen.head()
#Stromverbrauch.drop(columns=["Unnamed: 0"], inplace=True)
#Stromverbrauch.head()
#Wetterdaten.head()
#Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"}, inplace=True)
#Verkehrsdaten.head()




,Unnamed: 0,Datum,Hour,Traffic,Weekday
0,0,2022-01-01 00:00:00+00:00,0,1.0,Saturday
1,1,2022-01-01 00:00:00+00:00,1,0.0,Saturday
2,2,2022-01-01 00:00:00+00:00,2,0.0,Saturday
3,3,2022-01-01 00:00:00+00:00,3,0.0,Saturday
4,4,2022-01-01 00:00:00+00:00,4,0.0,Saturday


In [145]:
Parkhäuser.head()

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,Claramattechange,Elisabethenchange,Steinenchange,Kunstmuseumchange,Messechange,Anfoschange,Centralbahnparkingchange
0,2022-01-01 00:00:00+00:00,-7.0,-6.0,-15.0,-19.0,0.0,-2.0,-2.0,-3.0,-1.0,-23.0,-5.0,-10.0,-4.0,-7.0,0.0,-2.0
1,2022-01-01 01:00:00+00:00,-7.0,-1.0,-4.0,-9.0,-1.0,0.0,0.0,0.0,0.0,0.0,-9.0,-19.0,0.0,-8.0,-1.0,-1.0
2,2022-01-01 02:00:00+00:00,-4.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,-12.0,-23.0,0.0,-8.0,0.0,0.0
3,2022-01-01 03:00:00+00:00,-3.0,0.0,-1.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,-15.0,-10.0,0.0,-2.0,0.0,0.0
4,2022-01-01 04:00:00+00:00,-2.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,-1.0,-7.0,0.0,-2.0,0.0,1.0


In [146]:
Flugbewegungen.head()



,Unnamed: 0,Datum,Hour,Traffic,Weekday
0,0,2022-01-01 00:00:00+00:00,0,1.0,Saturday
1,1,2022-01-01 00:00:00+00:00,1,0.0,Saturday
2,2,2022-01-01 00:00:00+00:00,2,0.0,Saturday
3,3,2022-01-01 00:00:00+00:00,3,0.0,Saturday
4,4,2022-01-01 00:00:00+00:00,4,0.0,Saturday


In [147]:
Verkehrsdaten = Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"})
print(Verkehrsdaten['Datum'].isna().sum())


0


In [148]:
#standardise the data
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle

dfs = [Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten]

# Dictionary to store the mean and standard deviation for each dataframe
scalers = {}

# Loop through each dataframe
for i, df in enumerate(dfs):
    numeric_cols = df.select_dtypes(include=[np.number])
    if not numeric_cols.empty:
        if i == 1:  # Exclude 'Hour' column for Flugbewegungen
            numeric_cols = numeric_cols.drop(columns=['Hour'])
        print(f"Processing df{i} with columns: {numeric_cols.columns}")
        scaler = StandardScaler()
        # Fit the scaler on the dataframe and transform the data
        scaled_data = scaler.fit_transform(numeric_cols)
        
        # Save the mean and standard deviation
        scalers[f'df{i}'] = {'mean': scaler.mean_, 'scale': scaler.scale_}
        
        # Replace the original data with the standardized data
        df[numeric_cols.columns] = scaled_data
        dfs[i] = df
    else:
        print(f"df{i} has no numeric columns or is empty.")

# Assign the transformed dataframes back to their original names
Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten = dfs

# Save the scalers dictionary for later use
with open('scalers.pkl', 'wb') as f:
    pickle.dump(scalers, f)


Processing df0 with columns: Index(['Rebgassechange', 'Clarahuuschange', 'Citychange', 'Storchenchange',
       'Post Baselchange', 'Aeschenchange', 'Bahnhof Südchange',
       'Bad. Bahnhofchange', 'Europechange', 'Claramattechange',
       'Elisabethenchange', 'Steinenchange', 'Kunstmuseumchange',
       'Messechange', 'Anfoschange', 'Centralbahnparkingchange'],
      dtype='object')
Processing df1 with columns: Index(['Unnamed: 0', 'Traffic'], dtype='object')
Processing df2 with columns: Index(['Gasverbrauch'], dtype='object')
Processing df3 with columns: Index(['Stromverbrauch'], dtype='object')
Processing df4 with columns: Index(['Unnamed: 0', 'Basel Temperature [2 m elevation corrected]',
       'Basel Precipitation Total', 'Basel Wind Speed [10 m]',
       'Basel Wind Direction [10 m]'],
      dtype='object')
Processing df5 with columns: Index(['350n_sumPW', '350n_sumLief', '350n_sumLW', '350v_sumPW',
       '350v_sumLief', '350v_sumLW', '352n_sumPW', '352n_sumLief',
       '352

In [149]:
print(type(Verkehrsdaten))
Wetterdaten.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Gasverbrauch.sort_values(by=["Datum"], inplace=True)

<class 'pandas.core.frame.DataFrame'>


In [150]:
# Ensure 'Datum' is in datetime format
Flugbewegungen['Datum'] = pd.to_datetime(Flugbewegungen['Datum'])

# Add the 'Hour' column as timedelta
Flugbewegungen['Datum'] = Flugbewegungen['Datum'] + pd.to_timedelta(Flugbewegungen['Hour'], unit='h')

# Display the first 25 rows
Flugbewegungen.head(25)

,Datum,Hour,Traffic,Weekday
0,2022-01-01 00:00:00+00:00,0,-1.142151,Saturday
1,2022-01-01 01:00:00+00:00,1,-1.295479,Saturday
2,2022-01-01 02:00:00+00:00,2,-1.295479,Saturday
3,2022-01-01 03:00:00+00:00,3,-1.295479,Saturday
4,2022-01-01 04:00:00+00:00,4,-1.295479,Saturday
5,2022-01-01 05:00:00+00:00,5,-1.295479,Saturday
6,2022-01-01 06:00:00+00:00,6,-0.682167,Saturday
7,2022-01-01 07:00:00+00:00,7,-0.222182,Saturday
8,2022-01-01 08:00:00+00:00,8,-0.835495,Saturday
9,2022-01-01 09:00:00+00:00,9,-0.528839,Saturday


In [151]:
                                                                                                # The merge isnt correct because it erases the Hour format and therefore setting the entire Day to one value?!
# Merge all dataframes on the 'Datum' column
final_df = Parkhäuser.merge(Wetterdaten, on='Datum', how='inner') \
                      .merge(Stromverbrauch, on='Datum', how='inner') \
                      .merge(Verkehrsdaten, on='Datum', how='inner') \
                      .merge(Gasverbrauch, on='Datum', how='inner') \
#                      .merge(Flugbewegungen, on='Datum', how='inner') \

#                      .merge(Schallmessungen, on='Datum', how='inner') \ -> too many NaN values
# Display the first few rows of the merged dataframe
final_df.head(20)

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111104v_sumPW,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch
0,2022-01-01 00:00:00+00:00,-0.489850,-1.162123,-0.163777,-1.110018,-0.000078,-0.258065,-0.245463,-0.158260,-0.151263,...,-1.363950,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494
1,2022-01-01 01:00:00+00:00,-0.489850,-0.193864,-0.043944,-0.525850,-0.191327,-0.000173,-0.000199,-0.000464,-0.000215,...,-1.363950,0.745474,-0.744920,-0.955306,-0.781539,-1.055084,-1.048762,-0.773270,-1.072471,0.357518
2,2022-01-01 02:00:00+00:00,-0.279870,-0.000213,-0.011262,-0.058515,-0.000078,-0.000173,-0.122831,-0.000464,-0.000215,...,-1.363950,0.575505,-0.744920,-0.937353,-0.571421,-1.055084,-1.057758,-0.416962,-1.072471,0.385414
3,2022-01-01 03:00:00+00:00,-0.209877,-0.000213,-0.011262,-0.350599,-0.000078,-0.000173,-0.000199,-0.000464,-0.000215,...,-1.363950,0.269563,-0.744920,-1.045068,-0.571421,-1.055084,-1.201702,-0.416962,-1.072471,0.617131
4,2022-01-01 04:00:00+00:00,-0.139884,-0.000213,0.021420,0.116735,-0.000078,-0.000173,0.245064,-0.000464,0.150833,...,-1.363950,0.575505,-0.331206,-1.063021,-0.781539,-1.055084,-1.255681,-0.654501,-1.072471,1.109860
5,2022-01-01 05:00:00+00:00,0.140089,-0.000213,0.925614,-0.233766,-0.000078,0.128773,-0.000199,0.052135,-0.000215,...,-1.363950,0.915442,-0.538063,-1.036092,-0.781539,-1.055084,-1.147723,-0.535731,-1.072471,1.670331
6,2022-01-01 06:00:00+00:00,-0.069891,0.193439,-0.022156,-0.116932,-0.000078,0.128773,-0.122831,-0.000464,-0.151263,...,-1.363950,0.201576,-0.538063,-1.161759,-0.571421,0.004582,-1.255681,-0.773270,-1.072471,1.455102
7,2022-01-01 07:00:00+00:00,0.000102,-0.193864,0.054102,-0.058515,-0.000078,-0.000173,0.122433,0.052135,-0.151263,...,-1.363950,0.337550,-0.331206,-1.089949,-0.781539,-1.055084,-1.210699,-0.773270,-1.072471,1.472270
8,2022-01-01 08:00:00+00:00,0.000102,0.193439,0.141253,-0.000099,-0.000078,-0.000173,0.612960,-0.053063,-0.151263,...,-1.363950,0.813461,-0.744920,-1.000187,-0.676480,-0.525251,-1.048762,-0.654501,-0.611608,1.555734
9,2022-01-01 09:00:00+00:00,-0.069891,1.161698,0.173935,-0.175349,0.573670,-0.000173,0.612960,0.104733,-0.151263,...,-1.356259,1.391353,-0.331206,-0.919401,-0.256244,-0.525251,-0.976790,-0.535731,-0.611608,1.401258


In [156]:
final_df.tail(25)

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111104v_sumPW,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch
24565,2024-10-20 13:00:00+00:00,0.280075,-0.968471,-0.370761,0.467236,-0.000078,0.386665,-0.000199,-0.263458,-0.000215,...,0.212546,-0.206348,-0.538063,0.525775,0.269051,0.004582,0.372683,0.176884,-0.611608,-0.705971
24566,2024-10-20 14:00:00+00:00,-1.399761,-0.000213,-0.621321,-1.752602,-0.000078,-0.387011,2.697704,-0.842044,0.301880,...,0.097193,-0.206348,-0.538063,0.633490,-0.781539,0.004582,0.408669,0.414423,-0.150745,-0.716121
24567,2024-10-20 15:00:00+00:00,0.070096,-0.000213,-0.752048,-1.285268,-0.000078,-0.773850,-3.065999,-0.421254,0.452928,...,0.089503,-0.002386,-0.538063,0.875848,0.374110,0.004582,0.723545,0.176884,-0.150745,-0.654515
24568,2024-10-20 16:00:00+00:00,-0.489850,-0.000213,-0.338079,-0.701100,-0.000078,-0.773850,-1.226519,-0.736846,-0.151263,...,-0.125824,-0.172354,-0.538063,0.660418,0.163992,0.004582,0.462647,-0.298193,-0.611608,-0.551452
24569,2024-10-20 17:00:00+00:00,-0.279870,-0.000213,-0.152883,-0.642683,-0.000078,-0.515957,-1.103887,-0.842044,0.150833,...,-0.148894,-0.274335,-0.951777,0.148772,0.058933,-0.525251,0.003827,-0.060654,-0.150745,-0.463764
24570,2024-10-20 18:00:00+00:00,-0.209877,-0.000213,-0.283610,-0.525850,-0.000078,-0.000173,-0.858623,-0.526452,0.301880,...,-0.610308,-0.376316,-0.951777,-0.362873,-0.571421,-1.055084,-0.104131,0.295653,-1.072471,-0.471266
24571,2024-10-20 19:00:00+00:00,-0.209877,-0.000213,-0.120201,-1.285268,-0.000078,-0.129119,0.245064,-0.368655,-0.000215,...,-0.717971,-0.206348,-0.951777,-0.703971,-0.466362,-1.055084,-0.410012,0.176884,-1.072471,-0.521984
24572,2024-10-20 20:00:00+00:00,-0.069891,-0.968471,0.032314,-1.051601,-0.000078,-0.129119,-0.981255,-0.158260,-0.000215,...,-0.910226,-0.342323,-0.951777,-0.892472,-0.676480,-1.055084,-0.949801,-0.654501,-1.072471,-0.666680
24573,2024-10-20 21:00:00+00:00,0.000102,-0.000213,-0.468806,-0.233766,-0.000078,-0.000173,-0.245463,-0.000464,-0.151263,...,-1.117862,-0.410310,-0.951777,-1.045068,-0.781539,-1.055084,-1.183709,-0.535731,-1.072471,-0.789995
24574,2024-10-20 22:00:00+00:00,0.000102,-0.000213,-0.283610,-0.000099,-0.000078,-0.000173,-0.000199,-0.053063,-0.000215,...,-1.233216,-0.376316,-0.951777,-1.116878,-0.466362,-1.055084,-1.282671,-0.654501,-0.611608,-0.824848


In [153]:
test_dataFrame = [Parkhäuser,
                Wetterdaten,
                Flugbewegungen,
                Gasverbrauch,
                Stromverbrauch,
                Verkehrsdaten]

for i in test_dataFrame:
    print(i['Datum'].max())

2024-10-21 13:00:00+00:00
2024-11-22 23:00:00+00:00
2024-11-22 23:00:00+00:00
2024-11-24 04:00:00+00:00
2024-11-20 22:00:00+00:00
2024-11-19 22:00:00+00:00


# Check for rows with NaN values
nan_rows = final_df[final_df.isnull().any(axis=1)]

# Display the rows with NaN values
nan_rows.head()

In [157]:
type(final_df[["Datum"]])

pandas.core.frame.DataFrame

In [158]:
final_df.to_csv("../Working_DataFrame.csv", index=False)

In [ ]:

#final_df.tail(40)
final_df['Datum'].dtype
import numpy as np

cutoff = np.datetime64('2024-11-20 10:00:00')
final_df = final_df[final_df['Datum'] <= cutoff]
final_df.tail(40)

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111104v_sumPW,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch
24560,2024-10-20 08:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24561,2024-10-20 09:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24562,2024-10-20 10:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24563,2024-10-20 11:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24564,2024-10-20 12:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24565,2024-10-20 13:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24566,2024-10-20 14:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24567,2024-10-20 15:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24568,2024-10-20 16:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537
24569,2024-10-20 17:00:00,-0.139884,-0.000213,-0.076625,-0.467433,-0.000078,-0.129119,-0.000199,-0.000464,-0.000215,...,-1.233216,-0.444303,-0.744920,-1.089949,-0.781539,-1.055084,-1.219695,-0.77327,-1.072471,-0.773537


In [ ]:
#2 NaN in Stromverbrauch 2022-10-30 01:00:00 & 2023-10-29 01:00:00

# Verkehrsdatenset hat 5 NaN values 2024-10-20 22:00:00 & 2024-10-20 23:00:00 & 2024-10-21 00:00:00 & 2024-10-21 01:00:00 & 2024-10-21 02:00:00	

# Wetter hat 5 NaN values 2023-12-22 00:00:00 & 2023-12-22 01:00:00 & 2023-12-22 02:00:00 & 2023-12-22 03:00:00 & 2023-12-22 04:00:00
# habe nochmal nachgeschaut und es ist der ganze 22 und 23 Dezember 2023 || ist aber nur eine Auflösung die ich ausgewählt habe, sollte mit der anderen Auflösung nicht mehr vorkommen
